In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
import random

In [ ]:
def read_image():
    image_rgb = cv2.imread("image.tif", cv2.IMREAD_COLOR)
    image_rgb = cv2.cvtColor(image_rgb, cv2.COLOR_BGR2RGB)
    image_rgb = cv2.resize(image_rgb, (600, 600))
    return image_rgb

In [ ]:
image_rgb = read_image()
plt.imshow(image_rgb)
plt.show()

## Task 01: Point Processing


In [ ]:
def get_negative_rgb(image):
  return 255 - image

In [ ]:
image_rgb = read_image()
negative_rgb = get_negative_rgb(image_rgb)
plt.imshow(negative_rgb)
plt.show()

In [ ]:
def get_negative_hsv(image):
  image[:, :, 0] = (image[:, :, 0] + 90) % 180
  image[:, :, 2] = 255 - image[:, :, 2]
  return image

In [ ]:
image_rgb = read_image()
image_hsv = cv2.cvtColor(image_rgb, cv2.COLOR_BGR2HSV)
negative_hsv = get_negative_hsv(image_hsv)
plt.imshow(cv2.cvtColor(negative_hsv, cv2.COLOR_HSV2BGR))
plt.show()

**What differences are noticed between the RGB-negative and HSV-negative?**

The HSV-Negative is noticeably darker than the RGB-Negative.

## Task 02: Histogram Equalization

In [ ]:
def normalize(r):
    counts, bins = np.histogram(r, 256)
    cdf = counts.cumsum()
    cdf = (cdf - cdf.min()) * 255 / (cdf.max() - cdf.min())
    s = np.interp(r, bins[:-1], cdf)
    s = np.array(s, np.int32)
    return s

In [ ]:
def equalize_rgb(image):
  image[:, :, 0] = normalize(image[:, :, 0])
  image[:, :, 1] = normalize(image[:, :, 1])
  image[:, :, 2] = normalize(image[:, :, 2])
  return image

In [ ]:
image_rgb = read_image()
equalized_rgb = equalize_rgb(image_rgb)
plt.imshow(equalized_rgb)
plt.show()

In [ ]:
def equalize_hsv(image):
  image[:, :, 2] = normalize(image[:, :, 2])
  return image

In [ ]:
image_rgb = read_image()
image_hsv = cv2.cvtColor(image_rgb, cv2.COLOR_BGR2HSV)
equalized_hsv = equalize_hsv(image_hsv)
plt.imshow(cv2.cvtColor(equalized_hsv, cv2.COLOR_HSV2BGR))
plt.show()

**Does the HE result become same for both RGB and HSV models?**

No it does not.

For the RGB image, the color also get modified since intensity is coupled with color under the RGB model.

For the HSV image, the colour is not modified. We are able to modify just the intensity wihtout affecting colour, since the colour is not coupled with the intensity. However, there is a slight decrease in the saturation of the image, which can fixed by increasing the saturation by a fixed amount, as shown below.

In [ ]:
def fix_saturation(image, increase):
    max_vals = np.full(image[:, :, 1].shape, 255)
    image[:, :, 1] = np.minimum(image[:, :, 1] + increase, max_vals)
    return image

In [ ]:
image_rgb = read_image()
image_hsv = cv2.cvtColor(image_rgb, cv2.COLOR_BGR2HSV)
equalized_hsv = equalize_hsv(image_hsv)
fixed_hsv = fix_saturation(equalized_hsv, 10)
plt.imshow(cv2.cvtColor(fixed_hsv, cv2.COLOR_HSV2BGR))
plt.show()

## Task 03: Linear Filter

In [ ]:
def laplacian_response(inp):
    """ The laplacian filter, as specified below, is applied to the image. """
    
    filtr = np.array([[-1, -1, -1], 
                     [-1, 8, -1], 
                     [-1, -1, -1]])
    return cv2.filter2D(src=inp, ddepth=-1, kernel=filtr)

In [ ]:
def sharpen_rgb(image):
  image[:, :, 0] = image[:, :, 0] + laplacian_response(image[:, :, 0])
  image[:, :, 1] = image[:, :, 1] + laplacian_response(image[:, :, 1])
  image[:, :, 2] = image[:, :, 2] + laplacian_response(image[:, :, 2])
  return image

In [ ]:
image_rgb = read_image()
sharpened_rgb = sharpen_rgb(image_rgb)
plt.imshow(sharpened_rgb)
plt.show()

In [ ]:
def sharpen_hsv(image):
  image[:, :, 2] = image[:, :, 2] + laplacian_response(image[:, :, 2])
  return image

In [ ]:
image_rgb = read_image()
image_hsv = cv2.cvtColor(image_rgb, cv2.COLOR_BGR2HSV)
sharpened_hsv = sharpen_hsv(image_hsv)
plt.imshow(cv2.cvtColor(sharpened_hsv, cv2.COLOR_HSV2BGR))
plt.show()

Similar to histogram equalization, there is some colour changes noticeable in the RGB image. This is again due to the intensity being coupled with the colour under this model. The HSV image does not show this issue.

## Task 04: Non-Linear Filter

In [ ]:
def add_noise(inp):
    """ 1 pixel of the image is replaced with the value 255 and 1 pixel is replaced with the value 0 randomly 1000 * strength times. """
    
    x, y = inp.shape
    output = inp.copy()
    
    for i in range(20 * 1000):
        nx = random.randint(0, x - 1)
        ny = random.randint(0, y - 1)
        output[nx, ny] = 255
        nx = random.randint(0, x - 1)
        ny = random.randint(0, y - 1)
        output[nx, ny] = 0
    
    return output

In [ ]:
def add_noise_channel_wise(image):
  image[:, :, 0] = add_noise(image[:, :, 0])
  image[:, :, 1] = add_noise(image[:, :, 1])
  image[:, :, 2] = add_noise(image[:, :, 2])
  return image

In [ ]:
def pad_image(inp, border):
    """ Adds padding of size = border to every edge of the image. """
    
    x, y = inp.shape
    out = np.zeros((x + (border * 2), y + (border * 2)), dtype=int)
    out[border:-border, border:-border] = inp[:,:]
    return out

In [ ]:
def median_filter(inp, filter_size):
    """ The image is parsed section by section, with the section size specified by the filter size, and the central pixel of each section is replaced with the median value for the section. """
    
    x, y = inp.shape
    border = filter_size // 2
    padded = pad_image(inp, border)
    out = np.zeros([x, y])
    
    for i in range(border, x + border):
        for j in range(border, y + border):
            section = padded[i - border : i + border + 1, j - border : j + border + 1]
            out[i - border, j - border] = np.median(section)
    
    return out.astype(np.uint8)

In [ ]:
def median_filter_channel_wise(image, filter_size):
  image[:, :, 0] = median_filter(image[:, :, 0], filter_size)
  image[:, :, 1] = median_filter(image[:, :, 1], filter_size)
  image[:, :, 2] = median_filter(image[:, :, 2], filter_size)
  return image

In [ ]:
image_rgb = read_image()
noisy_rgb = add_noise_channel_wise(image_rgb)
plt.imshow(noisy_rgb)
plt.show()

In [ ]:
filtered_rgb = median_filter_channel_wise(noisy_rgb, 5)
plt.imshow(filtered_rgb)
plt.show()

In [ ]:
image_rgb = read_image()
image_hsv = cv2.cvtColor(image_rgb, cv2.COLOR_BGR2HSV)
noisy_hsv = add_noise_channel_wise(image_hsv)
plt.imshow(cv2.cvtColor(noisy_hsv, cv2.COLOR_HSV2BGR))
plt.show()

In [ ]:
filtered_hsv = median_filter_channel_wise(noisy_hsv, 5)
plt.imshow(cv2.cvtColor(filtered_hsv, cv2.COLOR_HSV2BGR))
plt.show()

Noise was added to all three channels for both the RGB and the HSV images. The median filter was also applied to all three channels in both cases. As can be seen, the outputs for both the RGB image and the HSV image are comparable.